In [10]:
import glob
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Cargar Dataset

El siguiente código itera entre los archivos .pkl que componen metadata_sitios busca filas que contengan "Pizza restaurant" y "Pizza delivery" en la columna "category" y las une a un nuevo dataset con el fin de hacer la carga del dataset de forma más eficiente

In [12]:
# Ruta de la carpeta con los archivos .pkl
carpeta = r"C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PF\PF_google_yelp\Proyecto-FInal\Proyecto-FInal\venv\Datasets\metadata_sitios_pkl"

# Obtener la lista de archivos .pkl en la carpeta
archivos_pkl = glob.glob(os.path.join(carpeta, "*.pkl"))

In [14]:
# Crear una lista para almacenar los resultados parciales
df_pizza_list = []

# Filtrar cada archivo por partes para evitar sobrecarga de memoria
for archivo in archivos_pkl:
    df = pd.read_pickle(archivo)  # Cargar archivo .pkl
    
    # Filtrar filas donde 'category' contenga "Pizza restaurant"
    df_pizza = df[df["category"].astype(str).str.contains("Pizza restaurant|Pizza delivery", case=False, na=False)]
    
    # Agregar los resultados a la lista
    df_pizza_list.append(df_pizza)

# Concatenar los resultados en un solo DataFrame
pizza_metadata = pd.concat(df_pizza_list, ignore_index=True)

pizza_metadata.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Primo Pizza & Subs,"Primo Pizza & Subs, 148 W St Joseph St, Easton...",0x89c46d5e4554eae1:0xa2f8b211524ca29a,None,40.682177,-75.209820,['Pizza delivery'],4.5,25,$$,"[[Thursday, 11AM–9PM], [Friday, 11AM–10PM], [S...","{'Service options': ['No-contact delivery', 'D...",Open ⋅ Closes 9PM,"[0x89c46c56cac9c661:0xdc9d0dd3f2b203ae, 0x89c4...",https://www.google.com/maps/place//data=!4m2!3...
1,Sir Pizza Xpress,"Sir Pizza Xpress, 201 E Broad St, Smithville, ...",0x8866cef69c5737a1:0xbdf838584dca0b9c,None,35.956944,-85.811111,['Pizza restaurant'],4.1,18,None,"[[Thursday, 11AM–9PM], [Friday, 11AM–10PM], [S...",{'Service options': ['Delivery']},Permanently closed,"[0x8866cef6ac8ffaf3:0x4ed8b2fb47c1be1a, 0x8866...",https://www.google.com/maps/place//data=!4m2!3...
2,Bucci's Greek & Italian Specialties,"Bucci's Greek & Italian Specialties, 8030 S Ho...",0x876c8410e0d7ec6d:0xb06492857fa9ec9d,Casual restaurant with an eclectic menu of piz...,39.572697,-104.922399,"['Italian restaurant', 'Pizza restaurant']",4.7,13,None,"[[Thursday, 11AM–9PM], [Friday, 11AM–9PM], [Sa...","{'Service options': ['Delivery'], 'Highlights'...",Permanently closed,None,https://www.google.com/maps/place//data=!4m2!3...
3,Dynamite Pizza & BBQ,"Dynamite Pizza & BBQ, 59069 Gratiot Ave, New H...",0x882518a534093375:0x4333d5f58f7af4c,None,42.735038,-82.785340,['Pizza restaurant'],4.1,8,None,"[[Thursday, 10AM–10PM], [Friday, 10AM–11PM], [...",{'Service options': ['Delivery']},Permanently closed,[0x882521d22bad192b:0x4c9cccde727dadfc],https://www.google.com/maps/place//data=!4m2!3...
4,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",0x89e3169821e62d4d:0x14ff0683c1ebca0e,None,42.559072,-70.881542,"['Pizza restaurant', 'Italian restaurant', 'De...",3.9,48,$$,"[[Thursday, 11AM–8PM], [Friday, 11AM–10PM], [S...","{'Service options': ['Takeout', 'Delivery'], '...",Permanently closed,"[0x89e31418f27b6a29:0x2fd2e82a6f96214c, 0x89e3...",https://www.google.com/maps/place//data=!4m2!3...


Creacion Dataset a usar

In [18]:
# Guardar como Parquet (más eficiente) y obtener su tamaño
pizza_metadata.to_parquet(r"C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PF\PF_google_yelp\Proyecto-FInal\Proyecto-FInal\datasets\S3\pizza_metadata.parquet")
print("Peso del Parquet:", os.path.getsize(r"C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PF\PF_google_yelp\Proyecto-FInal\Proyecto-FInal\datasets\S3\pizza_metadata.parquet") / (1024 * 1024), "MB")

Peso del Parquet: 7.2284746170043945 MB


# ETL

In [ ]:
# Eliminamos columnas que no serán utilizadas
pizza_metadata = pizza_metadata.drop(columns=['price', 'hours', 'MISC', 'state', 'relative_results', 'url', 'description', 'category'])

In [ ]:
# Eliminamos filas con id's duplicados
pizza_metadata = pizza_metadata.drop_duplicates(subset=['gmap_id'])

pizza_metadata['gmap_id'].duplicated().sum()